# แนะนำการออกแบบพรอมต์

การออกแบบพรอมต์คือกระบวนการสร้างและปรับแต่งพรอมต์สำหรับงานประมวลผลภาษาธรรมชาติ โดยจะต้องเลือกพรอมต์ที่เหมาะสม ปรับค่าพารามิเตอร์ และประเมินประสิทธิภาพของพรอมต์นั้น การออกแบบพรอมต์มีความสำคัญอย่างมากในการเพิ่มความแม่นยำและประสิทธิภาพของโมเดล NLP ในหัวข้อนี้ เราจะมาทำความเข้าใจพื้นฐานของการออกแบบพรอมต์โดยใช้โมเดลของ OpenAI เป็นตัวอย่างในการศึกษา


### แบบฝึกหัดที่ 1: การแบ่งคำ (Tokenization)
สำรวจการแบ่งคำโดยใช้ tiktoken ซึ่งเป็นตัวแบ่งคำที่รวดเร็วและเป็นโอเพ่นซอร์สจาก OpenAI  
ดูตัวอย่างเพิ่มเติมได้ที่ [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst)


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### แบบฝึกหัดที่ 2: ตรวจสอบการตั้งค่า Github Models Key

รันโค้ดด้านล่างเพื่อตรวจสอบว่า endpoint ของ Github Models ของคุณตั้งค่าเรียบร้อยแล้วหรือไม่ โค้ดนี้จะทดสอบด้วย prompt ง่าย ๆ และตรวจสอบผลลัพธ์ที่ได้ โดยอินพุต `oh say can you see` ควรจะถูกเติมต่อประมาณว่า `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### แบบฝึกหัดที่ 3: การแต่งเรื่อง

ลองดูว่าจะเกิดอะไรขึ้นเมื่อคุณขอให้ LLM ตอบกลับสำหรับคำถามเกี่ยวกับหัวข้อที่อาจไม่มีอยู่จริง หรือหัวข้อที่มันอาจไม่รู้จักเพราะอยู่นอกชุดข้อมูลที่ถูกฝึกไว้ (ข้อมูลใหม่กว่า) สังเกตว่าคำตอบจะเปลี่ยนไปอย่างไรถ้าคุณลองใช้คำถามที่ต่างออกไป หรือเปลี่ยนเป็นโมเดลอื่น


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### แบบฝึกหัดที่ 4: ตามคำสั่ง
ใช้ตัวแปร "text" เพื่อกำหนดเนื้อหาหลัก
และใช้ตัวแปร "prompt" เพื่อให้คำสั่งที่เกี่ยวข้องกับเนื้อหาหลักนั้น

ในที่นี้ เราขอให้โมเดลสรุปเนื้อหาให้กับนักเรียนชั้นประถมศึกษาปีที่ 2


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### แบบฝึกหัดที่ 5: คำขอที่ซับซ้อน
ลองส่งคำขอที่มีข้อความจากระบบ, ผู้ใช้ และผู้ช่วย
ระบบจะตั้งบริบทให้กับผู้ช่วย
ข้อความจากผู้ใช้และผู้ช่วยจะสร้างบริบทการสนทนาแบบหลายรอบ

สังเกตว่าบุคลิกของผู้ช่วยถูกตั้งให้ "ประชดประชัน" ในบริบทของระบบ
ลองใช้บริบทบุคลิกแบบอื่น หรือเปลี่ยนชุดข้อความระหว่างผู้ใช้และผู้ช่วยดู


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### แบบฝึกหัด: สำรวจสัญชาตญาณของคุณ
ตัวอย่างข้างบนนี้ให้รูปแบบที่คุณสามารถนำไปใช้สร้าง prompt ใหม่ๆ ได้ (เช่น แบบง่าย แบบซับซ้อน แบบคำสั่ง ฯลฯ) ลองสร้างแบบฝึกหัดอื่นๆ เพื่อสำรวจแนวคิดอื่นๆ ที่เราได้พูดถึง เช่น ตัวอย่าง คำใบ้ และอื่นๆ



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง โปรดทราบว่าการแปลโดยระบบอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
